In [1]:
import pandas as pd
import glob
import numpy as np

In [2]:
cornbelters_files = glob.glob("../KCLData/*.csv")
other_files = glob.glob("../CornBeltersData/*.csv")

all_files = cornbelters_files + other_files

# Read and concatenate all CSVs into one DataFrame
stuff_plus = [pd.read_csv(f) for f in all_files]
df = pd.concat(stuff_plus, ignore_index=True)

In [3]:
#Combine PitchCall KorBB and PlayResult into one column
def combined_columns(event):
    #If there is a value in KorBB return that since PlayResult sometimes does not have KorBB
    if not pd.isna(event['KorBB']):
        return event['KorBB']
    #If the ball is in play in pitchcall we want to return the result to map it
    if event['PitchCall'] == 'InPlay':
        return event['PlayResult']
    #Since we already have if there's a strikeout or if there's an inplay
    else:
        return event['PitchCall']

In [4]:
def validate_data(df):
    """Validate DataFrame columns and data types."""
    required_columns = ['GameID', 'Inning', 'PAofInning', 'Outs', 'OutsOnPlay', 'RunsScored', 'pitch_by_pitch', 'Balls', 'Strikes']
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Check for missing values
    for col in required_columns:
        if df[col].isna().any():
            print(f"Warning: {col} contains {df[col].isna().sum()} missing values")
    
    # Validate data types and ranges
    if not df['PAofInning'].apply(lambda x: pd.isna(x) or (isinstance(x, (int, float)) and x >= 1)).all():
        raise ValueError("PAofInning must be numeric and >= 1 or NaN")
    if not df['Outs'].isin([0, 1, 2, np.nan]).all():
        print(f"Warning: Invalid values in Outs. Expected 0, 1, 2, or NaN. Found: {df['Outs'].unique()}")
    if not df['OutsOnPlay'].apply(lambda x: pd.isna(x) or (isinstance(x, (int, float)) and x >= 0)).all():
        raise ValueError("OutsOnPlay must be numeric and >= 0 or NaN")
    if not df['RunsScored'].apply(lambda x: pd.isna(x) or (isinstance(x, (int, float)) and x >= 0)).all():
        raise ValueError("RunsScored must be numeric and >= 0 or NaN")
    if not df['Balls'].isin([0, 1, 2, 3, 4, np.nan]).all():
        print(f"Warning: Invalid values in Balls. Expected 0-4 or NaN. Found: {df['Balls'].unique()}")
    if not df['Strikes'].isin([0, 1, 2, np.nan]).all():
        print(f"Warning: Invalid values in Strikes. Expected 0-2 or NaN. Found: {df['Strikes'].unique()}")

In [5]:
def infer_base_state(df_inning):
    """Infer base state (None or Runners) based on prior plays in the inning."""
    if df_inning.empty:
        return pd.Series(['None'] * len(df_inning), index=df_inning.index)
    
    base_state = []
    runners_on = False
    df_inning = df_inning.sort_values('PAofInning')  # Ensure sorted order
    
    for i, (idx, row) in enumerate(df_inning.iterrows()):
        # Reset runners at the start of an inning or after 3 Outs
        runs_scored = 0
        runs_scored+=row['RunsScored'] if not pd.isna(row['RunsScored']) else 0
        if row['PAofInning'] == 1 or (i > 0 and df_inning.iloc[i-1]['Outs'] == 2):
            runners_on = False
        # Outcomes that likely add runners (non-Outs)
        if row['pitch_by_pitch'] in ['Single', 'Double', 'Triple', 'Walk', 'HitByPitch', 'Error', 'FieldersChoice', 'IntentionalWalk']:
            runners_on = True
        # Outcomes that clear bases or end inning
        elif row['pitch_by_pitch'] in ['HomeRun'] or (row['PAofInning'] - row['Outs'] - runs_scored == 0 ):
            runners_on = False
        base_state.append('Runners' if runners_on else 'None')
    
    return pd.Series(base_state, index=df_inning.index)

In [6]:
def create_run_expectancy_matrix(df):
    """
    Create a run expectancy matrix for college baseball data that includes count states.
    
    Parameters:
    df (pd.DataFrame): DataFrame with 'GameID', 'Inning', 'PAofInning', 'Outs', 'OutsOnPlay', 'RunsScored', 'pitch_by_pitch', 'Balls', 'Strikes'.
    
    Returns:
    pd.DataFrame: Run expectancy matrix with mean runs per base-out-count state.
    pd.DataFrame: Updated DataFrame with base_state and base_out_count_state columns.
    """
    # Validate input data
    validate_data(df)
    
    # Create a copy and clean data
    df = df.copy()
    df['GameID'] = df['GameID'].astype(str)
    df['Inning'] = df['Inning'].astype(str)
    df = df.dropna(subset=['GameID', 'Inning', 'PAofInning'])
    df['Outs'] = df['Outs'].fillna(0).astype(int)
    df['OutsOnPlay'] = df['OutsOnPlay'].fillna(0).astype(int)
    df['RunsScored'] = df['RunsScored'].fillna(0).astype(float)
    df['Balls'] = df['Balls'].fillna(0).astype(int).clip(0, 4)  # Ensure valid ball counts
    df['Strikes'] = df['Strikes'].fillna(0).astype(int).clip(0, 3)  # Ensure valid strike counts
    
    # Handle non-scoring pitches
    non_scoring = ['Foul', 'BallCalled', 'StrikeCalled', 'StrikeSwinging', 'FoulTip']
    df.loc[df['pitch_by_pitch'].isin(non_scoring), 'RunsScored'] = 0
    df.loc[df['pitch_by_pitch'] == 'StrikeoutSwinging', 'pitch_by_pitch'] = 'Strikeout'
    
    # Create inning identifier
    df['inning_id'] = df['GameID'] + '_' + df['Inning']
    
    # Reindex PAofInning to ensure sequential values
    df['PAofInning'] = df.groupby('inning_id').cumcount() + 1
    
    # Sort by inning and PAofInning
    df = df.sort_values(['inning_id', 'PAofInning'])
    
    # Infer base state for each pitch
    try:
        df['base_state'] = df.groupby('inning_id', group_keys=False).apply(infer_base_state)
    except Exception as e:
        print(f"Error in infer_base_state: {e}")
        for inning_id, group in df.groupby('inning_id'):
            if not group['PAofInning'].is_monotonic_increasing or group['PAofInning'].min() < 1:
                print(f"Problematic inning_id: {inning_id}")
                print(group[['PAofInning', 'Outs', 'OutsOnPlay', 'pitch_by_pitch', 'RunsScored']])
        raise
    
    # Create base-out-count state
    df['base_out_count_state'] = (df['base_state'] + '_' + 
                                 df['Outs'].astype(str) + '_' + 
                                 df['Balls'].astype(str) + '-' + 
                                 df['Strikes'].astype(str))
    
    # Calculate cumulative runs to end of inning
    runs_per_play = df.groupby(['inning_id', 'PAofInning', 'base_out_count_state'])['RunsScored'].sum().reset_index()
    runs_per_play['runs_to_end'] = runs_per_play.groupby('inning_id')['RunsScored'].transform(lambda x: x[::-1].cumsum()[::-1])
    
    # Compute mean run expectancy for each base-out-count state
    re_matrix = runs_per_play.groupby('base_out_count_state')['runs_to_end'].mean().reset_index()
    re_matrix.rename(columns={'runs_to_end': 'run_expectancy'}, inplace=True)
    
    return re_matrix, df

In [ ]:
def calculate_delta_run_exp(df, re_matrix):
    """
    Calculate delta run expectancy for each pitch event using OutsOnPlay.
    
    Parameters:
    df (pd.DataFrame): DataFrame with pitch data and base_out_count_state.
    re_matrix (pd.DataFrame): Run expectancy matrix.
    
    Returns:
    pd.DataFrame: DataFrame with added 'delta_run_exp' column.
    """
    df = df.copy()
    df['start_base_out_count_state'] = df['base_out_count_state']
    
    # Infer end state after pitch
    df['end_outs'] = df['Outs'] + df['OutsOnPlay']
    df['end_outs'] = df['end_outs'].clip(upper=3)
    df['end_balls'] = df['Balls'].copy()
    df['end_strikes'] = df['Strikes'].copy()
    df['end_base_state'] = df['base_state']
    
    # Update count based on pitch outcome
    # Balls increase ball count (unless it results in walk/HBP)
    df.loc[df['pitch_by_pitch'] == 'BallCalled', 'end_balls'] = (df['Balls'] + 1).clip(upper=4)
    
    # Strikes increase strike count (unless it results in strikeout)
    strike_outcomes = ['StrikeCalled', 'StrikeSwinging', 'FoulTip']
    df.loc[df['pitch_by_pitch'].isin(strike_outcomes), 'end_strikes'] = (df['Strikes'] + 1).clip(upper=3)
    
    # Foul balls increase strikes only if strikes < 2
    df.loc[(df['pitch_by_pitch'] == 'Foul') & (df['Strikes'] < 2), 'end_strikes'] = df['Strikes'] + 1
    df.loc[(df['pitch_by_pitch'] == 'Foul') & (df['Strikes'] >= 2), 'end_strikes'] = df['Strikes']
    
    # Reset count for plate appearance ending outcomes
    pa_ending_outcomes = ['HomeRun', 'Strikeout', 'Out', 'Sacrifice', 'Single', 'Double', 'Triple', 
                         'Walk', 'HitByPitch', 'Error', 'IntentionalWalk', 'FieldersChoice']
    df.loc[df['pitch_by_pitch'].isin(pa_ending_outcomes), 'end_balls'] = 0
    df.loc[df['pitch_by_pitch'].isin(pa_ending_outcomes), 'end_strikes'] = 0
    
    # Update base state for outcomes that clear or add runners
    df.loc[df['pitch_by_pitch'].isin(['HomeRun', 'Strikeout', 'Out', 'Sacrifice']), 'end_base_state'] = 'None'
    df.loc[df['pitch_by_pitch'].isin(['Single', 'Double', 'Triple', 'Walk', 'HitByPitch', 'Error', 'IntentionalWalk']), 'end_base_state'] = 'Runners'
    
    # Create end base-out-count state
    df['end_base_out_count_state'] = (df['end_base_state'] + '_' + 
                                     df['end_outs'].astype(str) + '_' + 
                                     df['end_balls'].astype(str) + '-' + 
                                     df['end_strikes'].astype(str))
    
    # Merge start and end run expectancy
    df = df.merge(re_matrix, left_on='start_base_out_count_state', right_on='base_out_count_state', how='left')
    df.rename(columns={'run_expectancy': 'start_re'}, inplace=True)
    df = df.merge(re_matrix, left_on='end_base_out_count_state', right_on='base_out_count_state', how='left')
    df.rename(columns={'run_expectancy': 'end_re'}, inplace=True)
    
    # Final fallback to 0 for any remaining missing values
    df['start_re'] = df['start_re'].fillna(0)
    df['end_re'] = df['end_re'].fillna(0)
    
    # Calculate delta run expectancy
    df['delta_run_exp'] = df['end_re'] - df['start_re'] + df['RunsScored']
    
    # Drop temporary columns
    temp_cols = ['start_re', 'end_re', 'base_out_count_state_x', 'base_out_count_state_y', 
                'end_outs', 'end_balls', 'end_strikes', 'end_base_state', 'end_base_out_count_state']
    # Only drop columns that exist
    temp_cols_to_drop = [col for col in temp_cols if col in df.columns]
    if 'start_base_out_state' in df.columns:
        temp_cols_to_drop.append('start_base_out_state')
    if 'end_base_out_state' in df.columns:
        temp_cols_to_drop.append('end_base_out_state')
    
    df.drop(temp_cols_to_drop, axis=1, inplace=True)
    
    return df

In [8]:
def calculate_mean_delta_run_exp(df):
    """
    Calculate mean delta run expectancy by pitch outcome and count.
    
    Parameters:
    df (pd.DataFrame): DataFrame with 'pitch_by_pitch', 'delta_run_exp', 'Balls', 'Strikes'.
    
    Returns:
    pd.DataFrame: DataFrame with mean delta_run_exp for each outcome and count.
    """
    mean_delta_run_exp = df.groupby(['pitch_by_pitch', 'Balls', 'Strikes','Outs'])['delta_run_exp'].mean().reset_index()
    mean_delta_run_exp.rename(columns={'delta_run_exp': 'delta_run_exp_mean'}, inplace=True)
    return mean_delta_run_exp

In [9]:
def build_delta_run_exp_df(df):
    """
    Build DataFrame with delta run expectancy for each pitch.
    
    Parameters:
    df (pd.DataFrame): DataFrame with 'GameID', 'Inning', 'PAofInning', 'Outs', 'OutsOnPlay', 'RunsScored', 
                       'pitch_by_pitch', 'Balls', 'Strikes'.
    
    Returns:
    pd.DataFrame: DataFrame with 'delta_run_exp' and 'delta_run_exp_mean' columns.
    pd.DataFrame: Run expectancy matrix.
    pd.DataFrame: Mean delta run expectancy by outcome and count.
    """
    # Create run expectancy matrix and update DataFrame with base states
    re_matrix, df_with_base_states = create_run_expectancy_matrix(df)
    
    # Calculate delta run expectancy for each pitch
    df_with_delta = calculate_delta_run_exp(df_with_base_states, re_matrix)
    
    # Calculate mean delta run expectancy by outcome and count
    mean_delta_run_exp_df = calculate_mean_delta_run_exp(df_with_delta)
    
    # Merge mean delta run expectancy back into DataFrame
    df_final = df_with_delta.merge(mean_delta_run_exp_df, on=['pitch_by_pitch', 'Balls', 'Strikes','Outs'], how='left')
    
    # Handle missing mean delta run expectancy values
    df_final['delta_run_exp_mean'] = df_final['delta_run_exp_mean'].fillna(0)
    
    return df_final, re_matrix, mean_delta_run_exp_df

In [10]:
df['PitchCall'].unique()

array(['Foul', 'BallCalled', 'StrikeCalled', 'StrikeSwinging',
       'HitByPitch', 'InPlay', nan, 'StrikeLooking', 'IntentionalWalk',
       'FoulTip'], dtype=object)

In [11]:
df['PlayResult'].unique()

array([nan, 'StrikeoutLooking', 'StrikeoutSwinging', 'Out', 'Single',
       'Walk', 'Error', 'Double', 'Triple', 'HomeRun', 'FieldersChoice',
       'Sacrifice', 'HItByPitch'], dtype=object)

In [12]:
df['pitch_by_pitch'] = df.apply(combined_columns,axis=1)

In [13]:
df = df.dropna(subset=['Outs','pitch_by_pitch','PAofInning','GameID'])

In [14]:
df['pitch_by_pitch'].unique()

array(['Foul', 'BallCalled', 'Strikeout', 'StrikeCalled',
       'StrikeSwinging', 'HitByPitch', 'Out', 'Walk', 'Single', 'Error',
       'Double', 'Triple', 'HomeRun', 'FieldersChoice', 'Sacrifice',
       'IntentionalWalk', 'StrikeoutSwinging', 'FoulTip'], dtype=object)

In [15]:

df['Strikes'].unique()

array([0, 1, 2])

In [16]:
df['OutsOnPlay'] = df['OutsOnPlay'].fillna(0)
df['RunsScored'] = df['RunsScored'].fillna(0)

In [17]:
df_final, re_matrix, mean_delta_run_exp_df = build_delta_run_exp_df(df)
df_final.to_csv('college_pitch_data_with_delta_run_exp.csv', index=False)
re_matrix.to_csv('college_run_expectancy_matrix.csv', index=False)
mean_delta_run_exp_df.to_csv('mean_delta_run_exp_by_outcome_count.csv', index=False)

C:\Users\isu_mvquirk\AppData\Local\Temp\ipykernel_12092\3571389530.py:42: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['base_state'] = df.groupby('inning_id', group_keys=False).apply(infer_base_state)


In [59]:
features = ['PlateLocHeight','PlateLocSide','pitch_by_pitch','strikes','balls','Outs']